# DNN

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress most of the logs
tf.get_logger().setLevel('WARNING')  # Adjust logging level

2023-12-26 14:30:48.411290: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-26 14:30:48.434146: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 14:30:48.434166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 14:30:48.434738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 14:30:48.438627: I tensorflow/core/platform/cpu_feature_guar

In [3]:
X = np.load('images64.npy')
y = np.load('styles64.npy')
# X and y are now numpy arrays containing your data

In [4]:
np.unique(y)

array(['Classic', 'Contemporary', 'Country', 'Minimalism', 'Modern',
       'Unique', 'Urban'], dtype='<U12')

In [5]:
from sklearn.calibration import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)



from tensorflow.keras.utils import to_categorical

# Assuming y contains integer labels
y = to_categorical(y, num_classes=7)

In [6]:
from sklearn.model_selection import train_test_split
 

# Now proceed with the train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

In [6]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(9400, 64, 64, 3) (1175, 64, 64, 3) (1175, 64, 64, 3)
(9400, 7) (1175, 7) (1175, 7)


In [7]:
X_train.shape[1:]

(64, 64, 3)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, AvgPool2D, Flatten, LeakyReLU

In [9]:
model = Sequential([
    Input(X_train.shape[1:]),
    
    Conv2D(filters=32, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=16, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Flatten(),
    
    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),
    
    Dense(7),
    Activation('softmax')
])

model.summary()

2023-12-26 14:28:09.446009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 14:28:10.789461: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-26 14:28:10.789591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        1568      
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 average_pooling2d (Average  (None, 16, 16, 32)        0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 16)        8208      
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 16)        6

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1,
                              min_lr=0.001)


In [11]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('start fitting')
history = model.fit(
    X_train, y_train,
    epochs=10000000,
    batch_size=32,
    validation_data=(X_val,y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

print('fitting done')

Num GPUs Available:  1
start fitting
Epoch 1/10000000


2023-12-26 14:28:16.265049: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-26 14:28:19.875587: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa54967f830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-26 14:28:19.875602: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-12-26 14:28:20.032481: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703568500.187270   41553 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


294/294 [==============================] - 9s 6ms/step - loss: 1.1223 - accuracy: 0.6821 - val_loss: 1.1572 - val_accuracy: 0.6970 - lr: 0.0010
Epoch 2/10000000
294/294 [==============================] - 1s 3ms/step - loss: 1.0159 - accuracy: 0.6974 - val_loss: 1.5063 - val_accuracy: 0.4077 - lr: 0.0010
Epoch 3/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.9688 - accuracy: 0.7076 - val_loss: 1.4129 - val_accuracy: 0.4774 - lr: 0.0010
Epoch 4/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.9255 - accuracy: 0.7170 - val_loss: 1.6519 - val_accuracy: 0.3702 - lr: 0.0010
Epoch 5/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.9002 - accuracy: 0.7254 - val_loss: 1.2211 - val_accuracy: 0.7132 - lr: 0.0010
Epoch 6/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.8774 - accuracy: 0.7282 - val_loss: 0.9117 - val_accuracy: 0.7404 - lr: 0.0010
Epoch 7/10000000
294/294 [=========================

In [12]:
y_pred = model.predict(X_test)
print(y_pred[0])

import numpy as np

y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels[:10]



37/37 [==============================] - 0s 1ms/step
[0.08987872 0.12068116 0.10877512 0.00846811 0.48212352 0.01987756
 0.17019583]


array([4, 4, 4, 4, 4, 4, 6, 4, 4, 4])

In [13]:
if y_test.ndim > 1:  # Check if y_test is one-hot encoded
    y_test_int_labels = np.argmax(y_test, axis=1)
else:
    y_test_int_labels = y_test  # y_test is already in integer label format

# Now calculate the accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test_int_labels, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test_int_labels, y_pred_labels))
print("Classification Report:\n", classification_report(y_test_int_labels, y_pred_labels))


Accuracy: 0.7285106382978723
Confusion Matrix:
 [[ 28   0   2   0  98   0   0]
 [  3   0   2   1  81   0   0]
 [  2   0  14   0  52   0   5]
 [  0   0   0   3  13   0   0]
 [ 13   0   2   2 797   0   4]
 [  0   0   3   0   4   0   0]
 [  0   0   0   0  32   0  14]]
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.22      0.32       128
           1       0.00      0.00      0.00        87
           2       0.61      0.19      0.29        73
           3       0.50      0.19      0.27        16
           4       0.74      0.97      0.84       818
           5       0.00      0.00      0.00         7
           6       0.61      0.30      0.41        46

    accuracy                           0.73      1175
   macro avg       0.44      0.27      0.30      1175
weighted avg       0.65      0.73      0.66      1175



/home/bae/miniconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bae/miniconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bae/miniconda3/envs/tf/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ML

In [23]:
X = np.load('images64.npy')
y = np.load('styles64.npy')
# X and y are now numpy arrays containing your data

In [27]:
from sklearn.calibration import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

# Reshape X from (len(X), 64, 64, 3) to (len(X), 1, 12288)
X = X.reshape(len(X),-1)  # -1 tells numpy to infer the dimension

from sklearn.model_selection import train_test_split

# Now proceed with the train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(9400, 12288) (1175, 12288) (1175, 12288)
(9400,) (1175,) (1175,)


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [29]:
# Random Forest
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

# Predictions
y_pred_RFC = RFC.predict(X_test)

# Accuracy
accuracy_RFC = accuracy_score(y_test, y_pred_RFC)
print(f"Accuracy: {accuracy_RFC * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_RFC = precision_score(y_test, y_pred_RFC, average='weighted')
recall_RFC = recall_score(y_test, y_pred_RFC, average='weighted')
f1_RFC = f1_score(y_test, y_pred_RFC, average='weighted')

print(f"Precision: {precision_RFC:.2f}")
print(f"Recall: {recall_RFC:.2f}")
print(f"F1 Score: {f1_RFC:.2f}")

# Confusion Matrix
conf_matrix_RFC = confusion_matrix(y_test, y_pred_RFC)
print("Confusion Matrix:")
print(conf_matrix_RFC)


Accuracy: 87.91%
Precision: 0.89
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 74   0   0   0  54   0   0]
 [  0  46   0   0  41   0   0]
 [  1   0  53   0  19   0   1]
 [  1   0   0  12   3   0   0]
 [  6   0   0   0 813   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [30]:
# SVM Classifier
SVM = SVC()

# Random Forest
SVM = RandomForestClassifier()
SVM.fit(X_train, y_train)

# Predictions
y_pred_SVM = SVM.predict(X_test)

# Accuracy
accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
print(f"Accuracy: {accuracy_SVM * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_SVM = precision_score(y_test, y_pred_SVM, average='weighted')
recall_SVM = recall_score(y_test, y_pred_SVM, average='weighted')
f1_SVM = f1_score(y_test, y_pred_SVM, average='weighted')

print(f"Precision: {precision_SVM:.2f}")
print(f"Recall: {recall_SVM:.2f}")
print(f"F1 Score: {f1_SVM:.2f}")

# Confusion Matrix
conf_matrix_SVM = confusion_matrix(y_test, y_pred_SVM)
print("Confusion Matrix:")
print(conf_matrix_SVM)



Accuracy: 88.09%
Precision: 0.89
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 75   0   0   0  53   0   0]
 [  1  46   0   0  40   0   0]
 [  1   0  53   0  20   0   0]
 [  0   0   0  12   4   0   0]
 [  5   0   0   0 814   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [31]:
# XGBClassifier
XGB= xgb.XGBClassifier()

XGB.fit(X_train, y_train)

# Predictions
y_pred_XGB = XGB.predict(X_test)

# Accuracy
accuracy_XGB = accuracy_score(y_test, y_pred_XGB)
print(f"Accuracy: {accuracy_XGB * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_XGB = precision_score(y_test, y_pred_XGB, average='weighted')
recall_XGB = recall_score(y_test, y_pred_XGB, average='weighted')
f1_XGB = f1_score(y_test, y_pred_XGB, average='weighted')

print(f"Precision: {precision_XGB:.2f}")
print(f"Recall: {recall_XGB:.2f}")
print(f"F1 Score: {f1_XGB:.2f}")

# Confusion Matrix
conf_matrix_XGB = confusion_matrix(y_test, y_pred_XGB)
print("Confusion Matrix:")
print(conf_matrix_XGB)



Accuracy: 87.83%
Precision: 0.89
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 75   0   0   0  53   0   0]
 [  0  47   0   1  39   0   0]
 [  1   0  53   0  19   0   1]
 [  1   0   0  12   3   0   0]
 [  7   1   1   0 810   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [32]:
# Naive Bayes
GNB= GaussianNB()

GNB.fit(X_train, y_train)

# Predictions
y_pred_GNB = GNB.predict(X_test)

# Accuracy
accuracy_GNB = accuracy_score(y_test, y_pred_GNB)
print(f"Accuracy: {accuracy_GNB * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_GNB = precision_score(y_test, y_pred_GNB, average='weighted')
recall_GNB = recall_score(y_test, y_pred_GNB, average='weighted')
f1_GNB = f1_score(y_test, y_pred_GNB, average='weighted')

print(f"Precision: {precision_GNB:.2f}")
print(f"Recall: {recall_GNB:.2f}")
print(f"F1 Score: {f1_GNB:.2f}")

# Confusion Matrix
conf_matrix_GNB = confusion_matrix(y_test, y_pred_GNB)
print("Confusion Matrix:")
print(conf_matrix_GNB)



Accuracy: 27.40%
Precision: 0.62
Recall: 0.27
F1 Score: 0.34
Confusion Matrix:
[[ 22  23   4  10  19  11  39]
 [ 13  21   1  12  18   9  13]
 [  7  16  20   3   9   3  16]
 [  1   3   0   3   0   1   8]
 [ 71  98  51 103 223 108 165]
 [  0   0   0   0   0   3   3]
 [  5   2   1   2   4   1  30]]
